# TP3 - Ejercicio 2
### Enunciado
a)  Dividir aleatoriamente el conjunto de datos en dos conjuntos, uno de entrenamiento y uno de prueba.

b)  Clasificar la variable categórica _sigdz_ que indica si el paciente posee o no una enfermedad coronaria, utilizando el método SVM. Calcular la matriz de confusión.

c)  Proponer  (y  tambien  realizar)  una  secuencia  de  pasos que  permita  encontrar  una clasificación  mejor  que  la  anterior,  evaluando  diferentes  valores  de C y  diferentes núcleos.

In [27]:
import datetime

from ml_tps.utils.dataframe_utils import divide_in_training_test_datasets
import pandas as pd
import os
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler
from ml_tps.utils.evaluation_utils import getConfusionMatrix, computeAccuracy

print("Import success")

Import success


In [36]:
dir_path = os.path.dirname(os.path.realpath("e2.ipynb"))
DEFAULT_FILEPATH = f"{dir_path}/../tp3/data/acath.xls"
DEFAULT_OBJECTIVE = "sigdz"
DEFAULT_TRAIN_PCTG = 0.6

print("Variable setting success")

Variable setting success


## Data set info

In [29]:
# a)  Divide dataset randomly into training and evaluation set
dataset = pd.read_excel(DEFAULT_FILEPATH)
dataset = dataset.dropna()      # TODO maybe deal with NaN otherwise?

datasetX = dataset.loc[:, dataset.columns != DEFAULT_OBJECTIVE]
scaler = MinMaxScaler()
datasetX_scaled = pd.DataFrame(scaler.fit_transform(datasetX), index=datasetX.index, columns=datasetX.columns)
dataset_scaled = pd.concat([datasetX_scaled, dataset[DEFAULT_OBJECTIVE]], axis=1)

train, test = divide_in_training_test_datasets(dataset_scaled, train_pctg=DEFAULT_TRAIN_PCTG)

X_train = train.loc[:, train.columns != DEFAULT_OBJECTIVE]
y_train = train[DEFAULT_OBJECTIVE]

X_test = test.loc[:, test.columns != DEFAULT_OBJECTIVE]
y_test = test[DEFAULT_OBJECTIVE]

print("Train size:", train.shape)
print("Test size:", test.shape)

*** No CODEPAGE record, no encoding_override: will use 'ascii'
Train size: (1354, 6)
Test size: (904, 6)


## SVM using RBF (radial basis function) kernel (default)

In [35]:
# b)  Classify categorical variable "sigdz" using default SVC SVM
words_then = datetime.datetime.now()
classifiers = list()
accuracies = list()
classifiers.append(svm.SVC(kernel='rbf', gamma='auto', C=1))        # using default parameters, written down for illustrative purposes
classifiers[0].fit(X_train, y_train)
predictions_test = pd.Series(classifiers[0].predict(X_test).T)
confusion_matrix = getConfusionMatrix(predictions_test, y_test)
accuracies.append(computeAccuracy(predictions_test, y_test))

words_now = datetime.datetime.now()
print("Training runtime: ", divmod((words_now - words_then).total_seconds(), 60))
print(classifiers[0])
print("\nConfusion Matrix:\n", confusion_matrix)
print("\nAccuracy:", accuracies[0])

Training runtime:  (0.0, 0.161669)
SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

Confusion Matrix:
 Actual     0.0  1.0
Predicted          
0.0        173   58
1.0        129  544

Accuracy: 0.793141592920354


## Parameter and kernel tuning

In [26]:
# c)  Evaluate different values for C and different nuclei to find better performing classifiers
clf = svm.SVC(kernel='poly', degree=3, C=1, cache_size=500)
clf.fit(X_train, y_train)
predictions_test2 = pd.Series(clf.predict(X_test).T)
confusion_matrix2 = getConfusionMatrix(predictions_test2, y_test)
accuracies.append(computeAccuracy(predictions_test2, y_test))

time_now = datetime.datetime.now()
print("====== SVM using poly kernel ======")
print(clf)
print("Runtime SVM2: ", divmod((time_now - words_now).total_seconds(), 60), "\n")
print("Confusion Matrix:\n", confusion_matrix2)
print("\nAccuracy:", accuracies[1])

